In [2]:
import pandas as pd
import re
import string
import nltk
pd.set_option('display.max_colwidth', 100)
import glob
stopwords = nltk.corpus.stopwords.words('english')

In [3]:
wn = nltk.WordNetLemmatizer()

In [4]:
path =r'C:\Users\Akshat\AppData\Roaming\nltk_data\corpora\movie_reviews\neg' # use your path
allFiles = glob.glob(path + "/*.txt")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_table(file_, header=None)
    list_.append(df)
frame = pd.concat(list_)

In [4]:
frame['label']=0

In [5]:
frame=frame.head(5000)

In [6]:
path =r'C:\Users\Akshat\AppData\Roaming\nltk_data\corpora\movie_reviews\pos' # use your path
allFiles = glob.glob(path + "/*.txt")
frame2 = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_table(file_, header=None)
    list_.append(df)
frame2 = pd.concat(list_)

In [7]:
frame2['label']=1

In [7]:
frame2=frame2.head(5000)

In [8]:
final_data=frame.append(frame2)

In [9]:
final_data.shape

(10000, 1)

In [ ]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

final_data['body_text_clean'] = final_data[0].apply(lambda x: remove_punct(x))

final_data.head()

In [14]:
final_data.rename(columns={0:'text'},inplace=True)

In [ ]:
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

final_data['body_text_reg'] = final_data['body_text_clean'].apply(lambda x: tokenize(x.lower()))
final_data.head()

In [ ]:
def stop(text):
    test=[wn.lemmatize(word) for word in text if word not in stopwords]
    return test
final_data['body_final'] = final_data['body_text_reg'].apply(lambda x: stop(x))
final_data.head()

In [11]:

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [15]:
s=clean_text(final_data['text'])

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_data[['text']], final_data['label'], test_size=0.2)


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['text'])
tfidf_train = tfidf_vect_fit.transform(X_train['text'])
tfidf_test = tfidf_vect_fit.transform(X_test['text'])

In [22]:
hv=HashingVectorizer(analyzer=clean_text,n_features=25000 ,non_negative=True,ngram_range=(2,2))
x_train_hv=hv.transform(X_train['text'])
x_train_hv_vec=pd.DataFrame(x_train_hv.toarray())
x_test_hv=hv.transform(X_test['text'])
x_test_hv_vec=pd.DataFrame(x_test_hv.toarray())

C:\Users\Akshat\Anaconda3\lib\site-packages\sklearn\feature_extraction\hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
C:\Users\Akshat\Anaconda3\lib\site-packages\sklearn\feature_extraction\hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [ ]:
reset_selective x_test_hv_vec

In [ ]:
X_train_vect =pd.DataFrame(tfidf_train.toarray())
X_test_vect =pd.DataFrame(tfidf_test.toarray())

In [18]:
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [ ]:
classifier.fit(X_train_vect, y_train)

y_pred = classifier.predict(X_test_vect)
#y_real_pred = classifier.predict(X_test_vect)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [23]:
classifier.fit(x_train_hv_vec, y_train)
y_pred_hv = classifier.predict(x_test_hv_vec)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_hv))

             precision    recall  f1-score   support

          0       0.71      0.67      0.69      1021
          1       0.68      0.72      0.70       979

avg / total       0.69      0.69      0.69      2000



In [31]:
reviews_new = ['This movie was excellent', 'Absolute joy ride', 
            'Steven Seagal was terrible', 'Steven Seagal shined through.', 
              'This was certainly a movie', 'Two thumbs up', 'I fell asleep halfway through', 
              "We can't wait for the sequel!!", '!', '?', 'I cannot recommend this highly enough', 
              'instant classic.', 'Steven Seagal was amazing. His performance was Oscar-worthy.','this movie is so bad, i ran away','amazing']

hash_vect_sample = HashingVectorizer(analyzer=clean_text,n_features=25000 ,non_negative=True,ngram_range=(2,2))
test = hash_vect_sample.transform(reviews_new)

C:\Users\Akshat\Anaconda3\lib\site-packages\sklearn\feature_extraction\hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [32]:
test_pred=classifier.predict(test)

In [33]:
for review, category in zip(reviews_new, test_pred):
    print('%r => %s' % (review, category))

'This movie was excellent' => 1
'Absolute joy ride' => 1
'Steven Seagal was terrible' => 0
'Steven Seagal shined through.' => 0
'This was certainly a movie' => 0
'Two thumbs up' => 1
'I fell asleep halfway through' => 0
"We can't wait for the sequel!!" => 0
'!' => 0
'?' => 0
'I cannot recommend this highly enough' => 1
'instant classic.' => 1
'Steven Seagal was amazing. His performance was Oscar-worthy.' => 0
'this movie is so bad, i ran away' => 0
'amazing' => 1


In [ ]:
rf=RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)
model=rf.fit(X_train_vect, y_train)
pred=model.predict(X_test_vect)
print(classification_report(y_test, pred))